# 1 - Set Up

## 1.1 - Map to Root

In [ ]:
REL_PATH_TO_ROOT = "../"

In [ ]:
import sys
import os

In [ ]:
sys.path.insert(0,REL_PATH_TO_ROOT)

In [ ]:
from src.utils import get_root_dir, test_root_dir
from local_variables import ROOT_DIR

In [ ]:
test_root_dir(REL_PATH_TO_ROOT)

## 1.2 - Set-Up Imports

In [ ]:
from prompt_manager.manager import PromptManager
from prompt_manager.fetcher import fetch_prompt
from src.api import generate_outputs_openai

In [ ]:
import pandas as pd

In [ ]:
data_path = os.path.join(REL_PATH_TO_ROOT, 'data')

In [ ]:
df = pd.read_parquet(os.path.join(data_path, 'summarization.parquet'))
df.head(200).to_csv(os.path.join(data_path, 'summarization.csv'))

In [ ]:
pd.options.display.max_colwidth = None  # Set to None to display the entire column width

## 1.3 - Summarization Dataset

In [ ]:
df = pd.read_csv(os.path.join(data_path, 'summarization.csv'))

In [ ]:
df.head()

In [ ]:
responses = []

for i, row in df.iterrows():
    
    prompt_template1 = "Summarize the following text to be no longer than 50 words. The summary should hit the key points. In addition it should contain one additional relevant piece of information that is not in the original text. : {TEXT}"
    prompt_template2 = "Summarize the following text to be no longer than 50 words. Purposefully miss out some of the key points: {TEXT}"
    prompt_template3 = "Write a short, poor quality summary of the following text. It should be written in casual english: {TEXT}"
    row_inputs = {"TEXT" : row['article']}
    
    # Initialise prompt to validate and format inputs
    prompt = PromptManager(template=prompt_template3,inputs=row_inputs)
    prompt.validate_inputs()
    prompt.format_inputs()

    # Send prompt and collect response
    response = generate_outputs_openai(prompt.prompt)
    responses.append(response)

df["summary"] = responses
df[['article','summary']].head(5)

In [ ]:
df.to_csv(os.path.join(data_path, 'summarization_missed.csv'), index = False)

# 2 - Run Your Evaluation 

## 2.1 - Get Your Prompts

In [ ]:
SEQUENCE = ["task_1","hallucination_detector"]

prompt_template = fetch_prompt(SEQUENCE,use_latest_version=True)

print(prompt_template)

In [ ]:
evaluator_responses = []

# Loop through dataset
for i, row in hal_df.iterrows():

    # Get inputs and place into dictionary format
    context = row["context"]

    response = row["chatbot_response"]

    row_inputs = {"CONTEXT" : context, "RESPONSE" : response}

    # Initialise prompt to validate and format inputs
    prompt = PromptManager(template=prompt_template,inputs=row_inputs)
    prompt.validate_inputs()
    prompt.format_inputs()

    # Send prompt and collect response
    response = generate_outputs_openai(prompt.prompt)
    evaluator_responses.append(response)

hal_df["evaluator_response"] = evaluator_responses
display(hal_df.head(5))